# Fit 1D Log-Gaussian Tuning Curves (Debug Notebook)

This notebook converts the `fit_tuning_curves` Snakemake rule into an interactive format for debugging.
It loads eccentricity-binned NSD synthetic data, filters by stimulus class (averaged across classes),
then fits a `LogGaussianTuningModel` to the spatial frequency tuning data for a single eccentricity bin.
Steps: (1) define wildcards and paths, (2) compute eccentricity bin labels, (3) load and filter data,
(4) initialize model and dataset, (5) fit the tuning curve, (6) inspect loss and model parameters.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.dirname(os.getcwd()))
from sfp_nsdsyn import tuning, utils

%load_ext autoreload
%autoreload 2

## Define Wildcards & Paths
Set all Snakemake wildcard values as Python variables and construct input/output file paths.

In [2]:
# Wildcards
subj = 'subj06'
dset = 'nsdsyn'
roi = 'V1'
vs = 'pRFcenter'
stim_class = 'avg'
e1, e2, enum = 0.5, 4, 7
curbin = 0
lr = 0.005
max_epoch = 1000  # production value: 8000

# Paths
OUTPUT_DIR = '/Volumes/server/Projects/sfp_nsd/derivatives/'
input_path = os.path.join(OUTPUT_DIR, 'dataframes', dset, 'binned',
                          f'e1-{e1}_e2-{e2}_nbin-{enum}_sub-{subj}_roi-{roi}_vs-{vs}.csv')

results_dir = os.path.join(OUTPUT_DIR, 'sfp_model', 'results_1D', dset)
fname_base = f'class-{stim_class}_lr-{lr}_eph-{max_epoch}_e1-{e1}_e2-{e2}_nbin-{enum}_curbin-{curbin}_sub-{subj}_roi-{roi}_vs-{vs}'
output_model = os.path.join(results_dir, f'model-params_{fname_base}.pt')
output_model_history = os.path.join(results_dir, f'model-history_{fname_base}.h5')
output_loss_history = os.path.join(results_dir, f'loss-history_{fname_base}.h5')

input_path

'/Volumes/server/Projects/sfp_nsd/derivatives/dataframes/nsdsyn/binned/e1-0.5_e2-4_nbin-7_sub-subj06_roi-V1_vs-pRFcenter.csv'

## Compute Eccentricity Bin Labels
Generate bin boundaries and labels using the eccentricity range and number of bins.

In [3]:
bin_list, bin_labels = tuning.get_bin_labels(e1, e2, enum=enum)
bin_labels

['0.5-1.0 deg',
 '1.0-1.5 deg',
 '1.5-2.0 deg',
 '2.0-2.5 deg',
 '2.5-3.0 deg',
 '3.0-3.5 deg',
 '3.5-4.0 deg']

## Load and Filter Data
Load the binned CSV, average across stimulus classes, and filter to the current eccentricity bin.

In [4]:
subj_df = pd.read_csv(input_path)

# stim_class == 'avg': average across stimulus classes
subj_df = subj_df.groupby(['sub', 'ecc_bin', 'vroinames', 'freq_lvl']).mean().reset_index()

# subj != 'avg', so use raw betas (not normed_betas)
beta_col = 'betas'

# Filter to current eccentricity bin
save_ecc_bin_name = bin_labels[curbin]
subj_df = subj_df.query('ecc_bin == @save_ecc_bin_name')

subj_df.head()

,sub,ecc_bin,vroinames,freq_lvl,betas,local_sf
0,subj06,0.5-1.0 deg,V1,0,1.871189,1.298769
1,subj06,0.5-1.0 deg,V1,1,2.039668,2.328555
2,subj06,0.5-1.0 deg,V1,2,1.956054,4.434276
3,subj06,0.5-1.0 deg,V1,3,1.653242,8.219168
4,subj06,0.5-1.0 deg,V1,4,0.645565,15.408550


## Initialize Model and Dataset
Create the LogGaussianTuningModel and wrap the data as a LogGaussianTuningDataset.

In [ ]:
my_model = tuning.LogGaussianTuningModel()
my_dataset = tuning.LogGaussianTuningDataset(subj_df['local_sf'], subj_df[beta_col])
my_model

model_file='/Volumes/server/Projects/sfp_nsd/derivatives/sfp_model/results_1D/nsdsyn/model-params_class-reverse-spiral_lr-0.005_eph-8000_e1-0.5_e2-4_nbin-7_curbin-6_sub-subj06_roi-V1_vs-pRFcenter.pt'

## Fit Tuning Curve
Run the optimization loop. Set `save_path=output_model` to persist the `.pt` file.

In [ ]:
loss_history, model_history = tuning.fit_tuning_curves(
    my_model, my_dataset,
    learning_rate=lr,
    max_epoch=max_epoch,  # production value: 8000
    print_every=200,
    save_path=None  # set to output_model to save
)
model_history.tail()

## Inspect Loss History
Plot the training loss over epochs to verify convergence.

In [ ]:
loss_history['ecc_bin'] = save_ecc_bin_name
model_history['ecc_bin'] = save_ecc_bin_name

fig, ax = plt.subplots(1, 1, figsize=(6, 3))
ax.plot(loss_history['loss'])
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title(f'Training Loss — {subj}, {roi}, bin {save_ecc_bin_name}')
plt.tight_layout()

## Inspect Model Parameters
View the final fitted parameters (slope, mode, sigma) from the last epoch.

In [ ]:
model_history.tail(1)